# Configuration of Indexing and Search




## Preprocessing

The input strings given to the insert and query functions are tokenized before they can be processed further. Depending on the type of documents to handle, different methods of tokenization can be more or less beneficial.

Available options are:

| Tokenize function | Description                                                |
|-------------------|------------------------------------------------------------|
| word_ngrams(n)    | Word n-grams of length n                                   |
| char_ngrams(n)    | Character n-grams of length n                              |
| char_ngrams(n, c) | Character n-grams of length n with a padding character "c" |
| custom            | A custom function                                          |


## Precision settings

Narrow-Down is based on Minhash Locality Sensitive Hashing as described in 
[Leskovec, Rajaraman and Ullman: “Mining of Massive Datasets”, Chapter 3.](http://infolab.stanford.edu/~ullman/mmds/book0n.pdf).

It is a heuristic method to search a body of sets to find the ones with a minimum Jaccard similarity to a given input set. Approximation happens on two levels:
1. To estimate the similarity between a pair of sets by calculating a number of hash functions (the Minhash algorithm)
2. To store the minhashes in a condensed way such that one can find items which share a number of common hashes (the LSH algorithm)

The algorithms can be tuned with a number of parameters which are based on the following configuration options:


| Parameter                | Default | Effect                                                                                                          |
|--------------------------|---------|-----------------------------------------------------------------------------------------------------------------|
| similarity_threshold     | 0.75    | The minimum Jaccard similarity every search result should have.                                                 |
| max_false_negative_proba | 0.05    | Probability of false negatives, i.e. that a result is not found although the similarity is above the threshold. |
| max_false_positive_proba | 0.05    | Probability of false positives, i.e. that a result is returned although the similarity is below the threshold.  |

Narrow-down automatically tries to set the internal algorithm parameters (e.g. number of hash permutations, number of rows and bands of the LSH) in a way that the given target probabilities can be reached with minimum CPU and memory consumption.


**Effect on precision, recall, memory, cpu, storage**
